Попробуем использовать модель для русского текста без дообучения

In [1]:
from transformers import pipeline

In [2]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [7]:
ARTICLE = """ В островном королевстве Мелнибонэ истово блюли древние обычаи, хотя вот уже пятьсот лет день за днем и час за часом теряла держава свое былое могущество и держалась теперь лишь торговлей с Молодыми Королевствами, да потому еще, что Призрачный Город Имррир служил местом встречи купцам со всего света. Много ли от тех обычаев пользы? Нельзя ли отринуть их и тем самым избежать уготованной судьбы?
Соперник императора Элрика предпочитал не задаваться такими вопросами. Он заявлял во всеуслышанье, что, пренебрегая обычаями предков, Элрик навлечет гибель на Мелнибонэ.
Вот начало той трагической истории, которая длилась не год и не два и за концом которой — гибель всего этого мира.
"""

In [8]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'Не  теряла держава   \xa0‘ператора’,   ‘‘’И\u2009’’  “”’ ‘Н.”  ””,  ’”. ”, ” ‘, ’, \u2009”,. \u2009, ‘.’. \u2009. “Н\u2009 “I’m sorry,” “'}]


Дообучим 

In [3]:
from datasets import DatasetDict, Dataset
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
import random

# 1. Загрузка и подготовка данных (2% от общего объема)
def load_sample_data(file_path, fraction=0.02):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return random.sample(lines, max(1, int(len(lines) * fraction)))

# Подготовка данных
train_samples = load_sample_data('train_text_ru.txt')
val_samples = load_sample_data('val_text_ru.txt')

def prepare_samples(samples):
    sources, targets = [], []
    for line in samples:
        parts = line.strip().split('</s>')
        if len(parts) >= 2:
            sources.append(parts[0].strip())
            targets.append(parts[1].replace('<s>', '').replace('</s>?', '').strip())
    return {'source': sources, 'target': targets}

dataset = DatasetDict({
    'train': Dataset.from_dict(prepare_samples(train_samples)),
    'validation': Dataset.from_dict(prepare_samples(val_samples))
})

# 2. Загрузка модели и токенизатора
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 3. Токенизация с уменьшенными параметрами
def tokenize_fn(examples):
    model_inputs = tokenizer(examples['source'], max_length=128, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=64, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_data = dataset.map(tokenize_fn, batched=True, batch_size=8)

# 4. Облегченное обучение
training_args = TrainingArguments(
    output_dir='./tmp',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # Всего 1 эпоха для демонстрации
    evaluation_strategy='steps',
    eval_steps=50,
    logging_steps=50,
    learning_rate=3e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

# 5. Краткое обучение (только 1 эпоха)
trainer.train()

# 6. Пример использования обученной модели
def generate_summary(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=64,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Тестовые примеры
test_texts = [
    "Научные исследования показали, что регулярные физические упражнения улучшают когнитивные функции.",
    "В последние годы искусственный интеллект достиг значительных успехов в обработке естественного языка."
]

# Генерация пересказов
for text in test_texts:
    summary = generate_summary(text, model, tokenizer)
    print(f"Исходный текст: {text}")
    print(f"Пересказ: {summary}\n")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?it/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 844.175, 'train_samples_per_second': 0.018, 'train_steps_per_second': 0.009, 'train_loss': 4.095225811004639, 'epoch': 1.0}
Исходный текст: Научные исследования показали, что регулярные физические упражнения улучшают когнитивные функции.
Пересказ: На физические упражнения регулярный ощерий терой пр

Исходный текст: В последние годы искусственный интеллект достиг значительных успехов в обработке естественного языка.
Пересказ:  обрапехой прегращен от о перпраментей горой ракон по



Возьмем две эпохи

In [1]:
from datasets import DatasetDict, Dataset
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
import random

# 1. Загрузка и подготовка данных (2% от общего объема)
def load_sample_data(file_path, fraction=0.02):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return random.sample(lines, max(1, int(len(lines) * fraction)))

# Подготовка данных
train_samples = load_sample_data('train_text_ru.txt')
val_samples = load_sample_data('val_text_ru.txt')

def prepare_samples(samples):
    sources, targets = [], []
    for line in samples:
        parts = line.strip().split('</s>')
        if len(parts) >= 2:
            sources.append(parts[0].strip())
            targets.append(parts[1].replace('<s>', '').replace('</s>?', '').strip())
    return {'source': sources, 'target': targets}

dataset = DatasetDict({
    'train': Dataset.from_dict(prepare_samples(train_samples)),
    'validation': Dataset.from_dict(prepare_samples(val_samples))
})

# 2. Загрузка модели и токенизатора
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 3. Токенизация с уменьшенными параметрами
def tokenize_fn(examples):
    model_inputs = tokenizer(examples['source'], max_length=128, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=64, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_data = dataset.map(tokenize_fn, batched=True, batch_size=8)

# 4. Облегченное обучение (2 эпохи)
training_args = TrainingArguments(
    output_dir='./tmp',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,  # Теперь 2 эпохи
    evaluation_strategy='steps',
    eval_steps=50,
    logging_steps=50,
    learning_rate=3e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

# 5. Обучение (2 эпохи)
trainer.train()

# 6. Пример использования обученной модели
def generate_summary(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=64,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Тестовые примеры
test_texts = [
    "Научные исследования показали, что регулярные физические упражнения улучшают когнитивные функции.",
    "В последние годы искусственный интеллект достиг значительных успехов в обработке естественного языка."
]

# Генерация пересказов
for text in test_texts:
    summary = generate_summary(text, model, tokenizer)
    print(f"Исходный текст: {text}")
    print(f"Пересказ: {summary}\n")

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/14 [00:00<?, ?it/s]

c:\Users\nicks\anaconda3\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 809.0544, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.017, 'train_loss': 3.604740687779018, 'epoch': 2.0}
Исходный текст: Научные исследования показали, что регулярные физические упражнения улучшают когнитивные функции.
Пересказ: На дерой полежить не поще на горония огропоренищичи про�

Исходный текст: В последние годы искусственный интеллект достиг значительных успехов в обработке естественного языка.
Пересказ: прещен огрения негороле наже дейстать по прочки должа



In [3]:
test_texts = [
    """ В островном королевстве Мелнибонэ истово блюли древние обычаи, хотя вот уже пятьсот лет день за днем и час за часом теряла держава свое былое могущество и держалась теперь лишь торговлей с Молодыми Королевствами, да потому еще, что Призрачный Город Имррир служил местом встречи купцам со всего света. Много ли от тех обычаев пользы? Нельзя ли отринуть их и тем самым избежать уготованной судьбы?
Соперник императора Элрика предпочитал не задаваться такими вопросами. Он заявлял во всеуслышанье, что, пренебрегая обычаями предков, Элрик навлечет гибель на Мелнибонэ.
Вот начало той трагической истории, которая длилась не год и не два и за концом которой — гибель всего этого мира.
"""
]

# Генерация пересказов
for text in test_texts:
    summary = generate_summary(text, model, tokenizer)
    print(f"Исходный текст: {text}")
    print(f"Пересказ: {summary}\n")

Исходный текст:  В островном королевстве Мелнибонэ истово блюли древние обычаи, хотя вот уже пятьсот лет день за днем и час за часом теряла держава свое былое могущество и держалась теперь лишь торговлей с Молодыми Королевствами, да потому еще, что Призрачный Город Имррир служил местом встречи купцам со всего света. Много ли от тех обычаев пользы? Нельзя ли отринуть их и тем самым избежать уготованной судьбы?
Соперник императора Элрика предпочитал не задаваться такими вопросами. Он заявлял во всеуслышанье, что, пренебрегая обычаями предков, Элрик навлечет гибель на Мелнибонэ.
Вот начало той трагической истории, которая длилась не год и не два и за концом которой — гибель всего этого мира.

Пересказ: Каго огроде неголаничение день за днем и час зе переже

